In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv

#first let's pick one company's info page at random and scrape from it the headers for our database
url = "https://www.organic-bio.com/en/company/12054-ALVARADO-STREET-BAKERY"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/106.0.0.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page = requests.get(url,headers=headers)
code = BeautifulSoup(page.content, "html.parser")
col_names = code.find_all('th')
col_names = [x.text for x in col_names]
col_names.insert(0,'category')
import csv
with open("database.csv", 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(col_names)

#now for every category let's get the basic url from which we'll get lists of all the pages of data it has
url = "https://www.organic-bio.com/en/directory/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/106.0.0.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page = requests.get(url,headers=headers)
code = BeautifulSoup(page.content, "html.parser")

categories = code.find_all('a')
categories = [x.text.strip() for x in categories]
categories = categories[47:167:3]

links = code.find_all('a', href=True)
links = [x['href'] for x in links]
links = links[47:165:3]
first_pages = []
for x in links:
    first_pages.append(url+x)

#at this point we have the link to the first page in each category
for del1 in range(len(categories)):
    category = categories[del1]
    url2= first_pages[del1]+'?groupService=46'
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/106.0.0.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page2 = requests.get(url2,headers=headers)
    code2 = BeautifulSoup(page2.content, "html.parser")

    data = code2.find_all('a', href=True)
    data = [x['href'] for x in data]

    data2 = data[len(data)-3]
    if '&' in data2:
        page_last = int(data2[6:data2.index('&')])
    else:
        page_last = 1

    final_links = []
    for i in range(page_last):
        final_links.append('?page='+str(i+1)+'&groupService=46')

    final_links2 = []
    for x in final_links:
        final_links2.append(first_pages[del1]+x)

#now we go through each of them getting a list of urls for all the pages each category has
    for del2 in range(len(final_links2)):
        url3= final_links2[del2]
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/106.0.0.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
        page3 = requests.get(url3,headers=headers)
        code3 = BeautifulSoup(page3.content, "html.parser")

        companies = code3.find_all('a', href=True)
        companies = [x['href'] for x in companies]

        companies2 = []
        for i in range(len(companies)):
            m = companies[i].startswith('https://www.organic-bio.com/en/company/')
            if m == True:
                companies2.append(companies[i])

#finally all we have to do is scrape the companies' data one by one from each url and add it as a new row to our csv
        for del3 in range(len(companies2)):
            url4 = companies2[del3]
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/106.0.0.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
            page4 = requests.get(url4,headers=headers)
            code4 = BeautifulSoup(page4.content, "html.parser")

            details = code4.find_all('td')
            details2 = [x.text.strip() for x in details]
            details2 = details2[3:len(details2)-1]
            details2.insert(0,category)

            with open("database.csv", 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(details2)